In [7]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [11]:
df = pd.read_csv('clean_data.csv')
df.head()

,Unnamed: 0,Date,Adj Close,Close,High,Low,Open,Volume,name,percent_movement
0,0,1999-11-18,26.511736,31.473534,35.765381,28.612303,32.546494,62546380.0,A,3.409086
1,1,1999-11-19,24.327528,28.880545,30.758226,28.478184,30.713518,15234146.0,A,6.346741
2,2,1999-11-22,26.511736,31.473534,31.473534,28.657009,29.551144,6577870.0,A,-6.107957
3,3,1999-11-23,24.101582,28.612303,31.205294,28.612303,30.400572,5975611.0,A,6.250000
4,4,1999-11-24,24.741781,29.372318,29.998213,28.612303,28.701717,4843231.0,A,-2.283105


In [ ]:
groups = df.groupby('name')